In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
from pymongo import MongoClient
from pprint import pprint

In [3]:
def initDB(name):
    client = MongoClient('localhost', 27017)
    return client[name]

In [4]:
def saveRow(row, client):
    collection = client['forex']
    collection.insert_one(row)

In [5]:
def printRows(client, limit=10):
    collection = client['forex']
    for item in collection.find().limit(limit):
        pprint(item)

In [6]:
def closeBanner(driver):
    try:
        banner = driver.find_element_by_class_name('popupCloseIcon')
        banner.click()
    except:
        pass

In [7]:
def setGMT(driver):
    elem = driver.find_element_by_id('economicCurrentTime')
    elem.click()
    elem = driver.find_element_by_id('liTz197')
    elem.click()

In [8]:
def applyDate(driver, begin, end):
    date_picker = driver.find_element_by_id('datePickerToggleBtn')
    date_picker.click()
    date_picker = driver.find_element_by_id('startDate')
    date_picker.clear()
    date_picker.send_keys(begin)
    date_picker = driver.find_element_by_id('endDate')
    date_picker.clear()
    date_picker.send_keys(end)
    date_picker = driver.find_element_by_id('applyBtn')
    date_picker.click()

In [9]:
def loadEvents(driver, client):
    events = driver.find_elements_by_xpath("//tr[@class='js-event-item']")
    for event in events:
        date = event.get_attribute('data-event-datetime')
        country = event.find_element_by_xpath(".//td[contains(@class,'flagCur')]/span").get_attribute('title')
        vol = event.find_element_by_xpath(".//td[contains(@class,'textNum')]").get_attribute('title')
        act = event.find_element_by_xpath(".//td[contains(@class,'act')]").text
        fore = event.find_element_by_xpath(".//td[contains(@class,'fore')]").text
        prev = event.find_element_by_xpath(".//td[contains(@class,'prev')]").text
        row = {'date': date, 'country': country, 'vol': vol, 'act': act, 'fore': fore, 'prev': prev}
        saveRow(row, client)

In [10]:
chrome_options = Options()
#chrome_options.add_argument('start-maximized')
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://ru.investing.com/economic-calendar/')

In [11]:
client = initDB('selenium')

In [12]:
closeBanner(driver)
setGMT(driver)

In [14]:
# тут можно будет сделать генератор дат
applyDate(driver, '01/03/1970', '28/03/1970')

In [17]:
loadEvents(driver, client)

In [18]:
printRows(client)

{'_id': ObjectId('5e2ccb713b3f21bfab553d56'),
 'act': '1,80%',
 'country': 'Великобритания',
 'date': '1970/03/01 09:00:00',
 'fore': ' ',
 'prev': '-2,60%',
 'vol': 'Умеренная волатильность'}
{'_id': ObjectId('5e2ccb713b3f21bfab553d57'),
 'act': '1,00%',
 'country': 'Великобритания',
 'date': '1970/03/01 09:00:00',
 'fore': ' ',
 'prev': '-0,40%',
 'vol': 'Низкая волатильность'}
{'_id': ObjectId('5e2ccb713b3f21bfab553d58'),
 'act': '8,00%',
 'country': 'Великобритания',
 'date': '1970/03/01 09:00:00',
 'fore': ' ',
 'prev': '8,00%',
 'vol': 'Высокая волатильность'}
{'_id': ObjectId('5e2ccb723b3f21bfab553d59'),
 'act': '1,60%',
 'country': 'Великобритания',
 'date': '1970/03/01 09:00:00',
 'fore': ' ',
 'prev': '-2,70%',
 'vol': 'Высокая волатильность'}
{'_id': ObjectId('5e2ccb723b3f21bfab553d5a'),
 'act': '0,80%',
 'country': 'Великобритания',
 'date': '1970/03/01 09:00:00',
 'fore': ' ',
 'prev': '-0,70%',
 'vol': 'Низкая волатильность'}
{'_id': ObjectId('5e2ccb733b3f21bfab553d5b'),


In [19]:
driver.quit()